### ISS VSE CA2 Part1

In [11]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import time
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import graph_io
from tensorflow.keras.models import load_model

# import numpy as np
from numpy import expand_dims
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from matplotlib import pyplot
from matplotlib.patches import Rectangle

In [13]:
img_path = 'raccoon-1.jpg'
model_path = './model'
model_fname = os.path.join(model_path, 'yolov3_model.h5')

In [14]:
input_w, input_h = 416, 416
image, image_w, image_h = load_image_pixels(img_path, (input_w, input_h))

In [16]:
model = load_model(model_fname)

D:\Anaconda3\envs\rtav\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [17]:
anchors = [[116,90, 156,198, 373,326], [30,61, 62,45, 59,119], [10,13, 16,30, 33,23]]

In [30]:
import tensorflow as tf
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io

# model_tiny = os.path.join(model_path, 'yolotiny.h5')

with tf.Session() as sess:
    tf.keras.backend.set_learning_phase(0)
    model = tf.keras.models.load_model(model_fname)
    constant_graph = graph_util.convert_variables_to_constants(
        sess,
        sess.graph.as_graph_def(),
        ["conv2d_10/BiasAdd", "conv2d_13/BiasAdd"])

graph_io.write_graph(constant_graph, "saved_model_pb", 
                    "yolo.pb", as_text=False)

AssertionError: conv2d_10/BiasAdd is not in graph

In [ ]:
# camera_port = 0
# camera = cv2.VideoCapture(camera_port)
# time.sleep(0.1)
# return_value, image = camera.read()
# cv2.imshow('image', image)
# del(camera)

In [19]:
tf.keras.backend.clear_session()

In [31]:
# model_name = './car_bus_train2.h5'
def freeze_graph(graph, session, output, save_pb_dir='.', save_pb_name='frozen_model.pb', save_pb_as_text=False):
    with graph.as_default():
        graphdef_inf = tf.graph_util.remove_training_nodes(graph.as_graph_def())
        graphdef_frozen = tf.graph_util.convert_variables_to_constants(session, graphdef_inf, output)
        graph_io.write_graph(graphdef_frozen, save_pb_dir, save_pb_name, as_text=save_pb_as_text)
        return graphdef_frozen

In [36]:
tf.keras.backend.set_learning_phase(0)

In [37]:
mmodel = load_model('mnist.h5')

TypeError: Cannot interpret feed_dict key as Tensor: Tensor Tensor("Placeholder_1531:0", shape=(3, 3, 1, 32), dtype=float32) is not an element of this graph.

In [23]:
session = tf.keras.backend.get_session()

In [32]:
INPUT_NODE = [t.op.name for t in model_tiny.inputs]
OUTPUT_NODE = [t.op.name for t in model_tiny.outputs]
print(INPUT_NODE, OUTPUT_NODE)
frozen_graph = freeze_graph(session.graph, session, [out.op.name for out in model.outputs])

AttributeError: 'str' object has no attribute 'inputs'

In [38]:
pb_file = './model/frozen_darknet_yolov3_model.pb'
output_dir = './model'
img_height = 416
input_shape = [1,img_height,img_height,3]
input_shape_str = str(input_shape).replace(' ','')
input_shape_str

'[1,416,416,3]'

In [39]:
mo_tf_path = '"C:\Program Files (x86)\IntelSWTools\openvino\deployment_tools\model_optimizer\mo_tf.py"'

In [40]:
%run -i {mo_tf_path} --input_model {pb_file} --output_dir {output_dir} --input_shape {input_shape_str} --data_type FP32

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	D:\Workspace\GitHub\ISS-VSE-2019-09-23-IS1FT-CA2-Part1\./model/frozen_darknet_yolov3_model.pb
	- Path for generated IR: 	D:\Workspace\GitHub\ISS-VSE-2019-09-23-IS1FT-CA2-Part1\./model
	- IR output name: 	frozen_darknet_yolov3_model
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	[1,416,416,3]
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input channels: 	False
TensorFlow specific parameters:
	- Input model in text protobuf format: 	False
	- Path to model dump for TensorBoard: 	None
	- List of shared libraries with TensorFlow custom layers implementation: 

In [48]:
import os
from PIL import Image
import numpy as np
import sys
sys.path.append('C:\Program Files (x86)\IntelSWTools\openvino_2019.2.275\python\python3.7')

In [49]:
try:
    from openvino import inference_engine as ie
    from openvino.inference_engine import IENetwork, IEPlugin
except Exception as e:
    exception_type = type(e).__name__
    print("The following error happened while importing Python API module:\n[ {} ] {}".format(exception_type, e))

The following error happened while importing Python API module:
[ ImportError ] DLL load failed: The specified module could not be found.


In [50]:
def pre_process_image(imagePath, img_height=224):
    # Model input format
    n, c, h, w = [1, 3, img_height, img_height]
    image = Image.open(imagePath)
    processedImg = image.resize((h, w), resample=Image.BILINEAR)

    # Normalize to keep data between 0 - 1
    processedImg = (np.array(processedImg) - 0) / 255.0

    # Change data layout from HWC to CHW
    processedImg = processedImg.transpose((2, 0, 1))
    processedImg = processedImg.reshape((n, c, h, w))

    return image, processedImg, imagePath

In [51]:
# Plugin initialization for specified device and load extensions library if specified.
plugin_dir = None
model_xml = './model/frozen_darknet_yolov3_model.xml'
model_bin = './model/frozen_darknet_yolov3_model.bin'
# Devices: GPU (intel), CPU, MYRIAD
plugin = IEPlugin("MYRIAD", plugin_dirs=plugin_dir)
# Read IR
net = IENetwork(model=model_xml, weights=model_bin)
assert len(net.inputs.keys()) == 1
assert len(net.outputs) == 1
input_blob = next(iter(net.inputs))
out_blob = next(iter(net.outputs))
# Load network to the plugin
exec_net = plugin.load(network=net)
del net

NameError: name 'IEPlugin' is not defined

In [59]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise Exception("Could not open video device")
    
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 416)
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 416)
# print(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# print(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = cv2.resize(frame, (416,416))
    # Display the resulting frame
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()